# Characterizing CpG Methylation

To describe general metylation trends, irrespective of pCO<sub>2</sub> treatment in *C. virginica* gonad sequence data, I need to characterize individual CpG loci. Gavery and Roberts (2013) and Olson and Roberts (2013) define a CpG locus as methylated if at least half of the reads remained unconverted after bisulfite treatment. I will use information in `.cov` files to identify methylated CpG loci.

1. Download coverage files
2. Limit to 5x coverage only
3. Concatenate 5x loci for all samples
4. Identify loci that are 50% methylated

## 0. Prepare for analyses

## 0a. Set working directory

In [1]:
pwd

'/Users/yaamini/Documents/yaamini-virginica/notebooks'

In [2]:
cd ../analyses/

/Users/yaamini/Documents/yaamini-virginica/analyses


In [3]:
!mkdir 2019-03-18-Characterizing-CpG-Methylation

In [4]:
cd 2019-03-18-Characterizing-CpG-Methylation/

/Users/yaamini/Documents/yaamini-virginica/analyses/2019-03-18-Characterizing-CpG-Methylation


## 1. Obtain coverage files

In [5]:
#Download files from gannet. The files will be downloaded in the same directory structure they are in online.
!wget -r -l1 --no-parent -A.deduplicated.bismark.cov.gz \
http://gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2018-11-07-Bismark-Mox/

--2019-03-18 16:16:05--  http://gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2018-11-07-Bismark-Mox/
Resolving gannet.fish.washington.edu... 128.95.149.52
Connecting to gannet.fish.washington.edu|128.95.149.52|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: 'gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2018-11-07-Bismark-Mox/index.html'

gannet.fish.washing     [ <=>                ]  61.14K  --.-KB/s    in 0.001s  

2019-03-18 16:16:07 (45.1 MB/s) - 'gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2018-11-07-Bismark-Mox/index.html' saved [62605]

Loading robots.txt; please ignore errors.
--2019-03-18 16:16:07--  http://gannet.fish.washington.edu/robots.txt
Reusing existing connection to gannet.fish.washington.edu:80.
HTTP request sent, awaiting response... 404 Not Found
2019-03-18 16:16:07 ERROR 404: Not Found.

Removing gann

In [6]:
#Move all files from gannet folder to the current directory
!mv gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2018-11-07-Bismark-Mox/* .

In [7]:
#Confirm all files were moved
!ls

@eaDir
gannet.fish.washington.edu
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_1_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_2_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_3_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_4_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_5_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_6_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_7_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_8_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz
zr2096_9_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov.gz


In [8]:
#Remove the empty gannet directory
!rm -r gannet.fish.washington.edu

In [9]:
#Unzip the coverage files
!gunzip *cov.gz

In [10]:
#Confirm files were unzipped
!ls *cov

zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_1_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_2_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_3_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_4_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_5_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_6_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_7_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_8_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_9_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov


In [14]:
#Remove samples from high pCO2 treatment
!rm zr2096_6_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov \
zr2096_7_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov \
zr2096_8_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov \
zr2096_9_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov \
zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov

In [15]:
#Confirm file removal
!ls *cov

zr2096_1_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_2_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_3_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_4_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov
zr2096_5_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov


In [13]:
#See what the file looks like. 
#Columns: <chromosome> <start position> <end position> <methylation percentage> <count methylated> <count unmethylated>
!head -n 1 zr2096_10_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov

NC_007175.2	49	49	0	0	5


## 2. Limit to 5x coverage

For each coverage file, I want to retain all loci that have 5x coverage only. Using `awk`, I'll add the count methylated and unmethylated to get coverage. If that coverage is higher than 5, I'll redirect that information into a new file.

In [18]:
%%bash
for f in *.cov
do
    awk '{print $1, $2-1, $2, $4, $5+$6}' ${f} | awk '{if ($5 >= 5) { print $1, $2-1, $2, $4 }}' \
> ${f}_5x.bedgraph
done

In [21]:
#Confirm files were created
!ls *5x.bedgraph

zr2096_1_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov_5x.bedgraph
zr2096_2_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov_5x.bedgraph
zr2096_3_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov_5x.bedgraph
zr2096_4_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov_5x.bedgraph
zr2096_5_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov_5x.bedgraph


In [20]:
#Check columns for one of the file
!head zr2096_1_s1_R1_val_1_bismark_bt2_pe.deduplicated.bismark.cov_5x.bedgraph

NC_007175.2 1579 1580 0
NC_007175.2 2180 2181 0
NC_007175.2 3383 3384 0
NC_007175.2 3394 3395 0
NC_007175.2 5413 5414 0
NC_007175.2 5415 5416 0
NC_007175.2 5426 5427 0
NC_007175.2 11101 11102 0
NC_007175.2 12881 12882 0
NC_007175.2 12985 12986 20


## 3. Concatenate 5x loci for all samples

Now that I know how many loci have at least 5x coverage in each control sample, I want to see which loci have 5x coverage across all samples.